<a href="https://colab.research.google.com/github/heyhiheyhi/personal/blob/master/Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from tensorflow.keras.datasets.cifar10 import load_data

In [0]:
train_epochs = 10000
batch_size = 128
display_step = 100
test_epochs = 10
test_batch_size = 1000

In [0]:
#랜덤형태로 셔플 후 가져옴
def next_batch(num, data, labels):
  ''''num' 갯수만큼 랜덤하게 샘플 이미지와 레이블을 리턴합니다.'''
  idx = np.arange(0, len(data))
  np.random.shuffle(idx)
  idx=idx[:num]
  data_shuffle = [data[i] for i in idx]   #이미지 데이터
  labels_shuffle = [labels[i] for i in idx] #레이블 데이터
  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
def build_CNN_classifier(x):
  x_image = x

  #1st convolution layer
  #64filter 5x5x3
  #32x32x3 --> 32x32x64
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64], stddev=5e-2)) #웨이트값 5*5
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding="SAME")+b_conv1)
  
  #1st pooling layer
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding="SAME")

  #2nd convolution layer
  #64 filter 5x5
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding="SAME")+b_conv2)
  
  #2nd pooling layer
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding="SAME")

  #3rd convolution layer
  #128 filter 3x3
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3,3,64,128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1,1,1,1], padding="SAME")+b_conv3)
  
  #4rd convolution layer
  #128 filter 3x3
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1,1,1,1], padding="SAME")+b_conv4)
  
  #5th convolution layer
  #128 filter 3x3
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1,1,1,1], padding="SAME")+b_conv5)  

  #Fully connected layer1
  #32x32 -> 8x8x128 feature map -> 384 feature map
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*128,384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat,W_fc1)+b_fc1)
  
  #dropout
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  #Fully connected layer2
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384,10], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

  logits = tf.matmul(h_fc1_drop, W_fc2)+b_fc2

  #softmax
  y_pred = tf.nn.softmax(logits)

  return y_pred,logits

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 32,32,3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

AttributeError: ignored

In [0]:
#load_data를 실행하면서 다운로드가 진행된다.
(x_train,y_train),(x_test,y_test) = load_data()
#scalar 레이블을 one_hot encoding으로 변환
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis=1)

In [0]:
y_pred, logits = build_CNN_classifier(x)

In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [0]:
correct_prediction = tf.equal(tf.argmax(y_pred, 1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  # training & accuracy display
  for i in range(train_epochs):
    batch_x, batch_y = next_batch(batch_size, x_train, y_train_one_hot.eval())

    #display_step 마다 loss값 출력한다.
    if i%100==0:
      train_accuracy=accuracy.eval(feed_dict={x:batch_x, y:batch_y, keep_prob:1.0})
      loss_print=loss.eval(feed_dict={x:batch_x, y:batch_y, keep_prob:1.0})
      print("epoch:%d,accuracy:%f, loss:%f" %(i, train_accuracy, loss_print))
    sess.run(train_step, feed_dict={x:batch_x, y:batch_y, keep_prob:0.8})

  # test
  test_accuracy = 0.0
  for i in range(test_epochs):
    test_batch_x, test_batch_y = next_batch(test_batch_size, x_test, y_test_one_hot.eval())
    test_accuracy=test_accuracy + accuracy.eval(feed_dict={x:test_batch_x, y:test_batch_y, keep_prob:1.0})
    test_accuracy=test_accuracy / test_epochs
    print("test accuracy : %f" %test_accuracy)
